## Linear Regression with PyTorch

In [1]:
import numpy as np
import torch

In [2]:
#training data
#inputs -> temp, humidity, rainfall
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 76, 90]], dtype='float32')

#targets -> apples, oranges
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [3]:
#inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

#### Model

In [4]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.6654,  0.8732,  0.6607],
        [-1.8005, -1.8065, -1.1030]], requires_grad=True)
tensor([0.5065, 1.1299], requires_grad=True)


In [5]:
#model
def model(x):
    return x @ w.t() + b        #@ -> matrix multiplication in pytorch; t-> transpose

In [6]:
#predictions
preds = model(inputs)
print(preds)

tensor([[ 135.9995, -298.7733],
        [ 180.1900, -392.2823],
        [ 213.7316, -461.5628],
        [ 130.3754, -301.0135],
        [ 172.2511, -359.6703]], grad_fn=<AddBackward0>)


In [7]:
#compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [8]:
#loss function (mse)
def mse(t1, t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [9]:
loss = mse(preds, targets)
loss.sqrt()

tensor(334.3588, grad_fn=<SqrtBackward0>)

In [10]:
#gradient descent
loss.backward()

In [11]:
print(w)
print(w.grad)

tensor([[ 0.6654,  0.8732,  0.6607],
        [-1.8005, -1.8065, -1.1030]], requires_grad=True)
tensor([[  7788.1045,   7341.1899,   5105.0127],
        [-38208.3516, -39740.3203, -27499.7578]])
